# Analysis of Vienna Night Run Results

## Data Cleaning

In [1]:
import pandas as pd

Read raw data and add column names.

In [2]:
year = 2022
path_to_raw = "data/raw/vnr-results-public-" + str(year) + ".txt"
raw_data = pd.read_csv(path_to_raw, delimiter='\t', header=None, na_values=" ")
raw_data.drop(axis=1, columns=[0, 8], inplace=True)
raw_data.columns = ["id", "name", "sex", "company", "mins_per_km", "km_per_hour", "time"]
z=raw_data

In [3]:
z.head()

,id,name,sex,company,mins_per_km,km_per_hour,time
0,2979,Dominik Stadlmann,m,Tony's Laufshop,2:52,20.8,"14:23,6"
1,4178,Dominik Jandl,m,adesso,2:54,20.6,"14:31,5"
2,2342,Jan Michael Ratay,m,LCA Umdasch Amstetten,3:00,19.9,"15:00,6"
3,3218,Indrek Tobreluts,m,Spordiklubi TT,3:02,19.7,"15:10,5"
4,2952,Philipp Gintenstorfer,m,Hawi'd Ehre,3:03,19.6,"15:17,4"


Calculate time attributes.

In [13]:
# Keep only times less than 60 min
z = z[z["time"].str.split(":").str.len() == 2]
# z.head()
# Split minutes part from the rest
time_split = z["time"].str.split(":")
# Populate minutes column
z["mins"] = pd.Series([x[0] for x in time_split]).astype(int)
z["rest"] = pd.Series([x[1] for x in time_split])
# Split seconds from the tenths of second
secs_split = z["rest"].str.split(",")
# Populate seconds column
z["secs"] = pd.Series([x[0] for x in secs_split]).astype(int)
# Populate tenth of seconds column
z["tsecs"] = pd.Series([x[1] for x in secs_split]).astype(int)
# Calculate total time in seconds
z["time_secs"] = z["mins"] * 60 + z["secs"] + z["tsecs"] * 0.1
# Calculate total time in minutes
z["time_mins"] = z["time_secs"] / 60
# Calculate pace (seconds per km)
z["secs_per_km_calc"] = z["time_secs"] / 5
z.head()

,id,name,sex,company,mins_per_km,km_per_hour,time,mins,rest,secs,tsecs,time_secs,time_mins,secs_per_km_calc
0,2979,Dominik Stadlmann,m,Tony's Laufshop,2:52,20.8,"14:23,6",14,"23,6",23,6,863.6,14.393333,172.72
1,4178,Dominik Jandl,m,adesso,2:54,20.6,"14:31,5",14,"31,5",31,5,871.5,14.525000,174.30
2,2342,Jan Michael Ratay,m,LCA Umdasch Amstetten,3:00,19.9,"15:00,6",15,"00,6",0,6,900.6,15.010000,180.12
3,3218,Indrek Tobreluts,m,Spordiklubi TT,3:02,19.7,"15:10,5",15,"10,5",10,5,910.5,15.175000,182.10
4,2952,Philipp Gintenstorfer,m,Hawi'd Ehre,3:03,19.6,"15:17,4",15,"17,4",17,4,917.4,15.290000,183.48


Fix categorical factors.

Confirm pace as (incorrectly) provided in the original dataset.

Calculate correct speed.

Keep only relevant columns.

Save cleaned results.